In [1]:
from bs4 import BeautifulSoup as bs
import urllib
import requests
import pandas as pd
import re
import os
import sys
import json
import tqdm

In [2]:
# 네이버 API 로그인 후 link 정보 가져와 돌려주기
def get_Nav_client(encText, sPage):

    url = "https://openapi.naver.com/v1/search/kin?query=" + encText # json 결과
    url = url + "&start=" + str(sPage) + "&display=100"

    headers = {"X-Naver-Client-Id":"56fdCdrw5yUynfpm88Xr", 
               "X-Naver-Client-Secret":"aeN6nOYHvM"}
    
    # 네이버 API 연결(로그인)
    res_content=requests.get(url, headers=headers)

    if res_content.status_code == 200:
        res_json = res_content.json()
    else:
        print("Error Code:" + str(res_json.status_code))
        sys.exit(0)

    df=pd.DataFrame(res_json['items']) # 기본 DataFrame DataFrame 생성
    
    df_body=get_write(list(df['link'])) # 질문/답변 DataFrame 받아오기
    
    df=pd.concat([df, df_body], axis=1)
    #display(df.head(3))   
    return df

In [3]:
#link를 이용해 세부 내용 가져와 파일에 기록하기
def get_write(reg_link):
    
    body_lst = []
    
    for link in reg_link:
        kin_html = requests.get(link)

        if kin_html.status_code != 200 :
            print("Error Code:" + str(res_json.status_code))
            body_lst.append({"질문":"질문없음", "답변":"답변없음"})
            
        else:
            kin_soup = bs(kin_html.text, "html.parser")

            # 질문 내용
            try:
                kin_body=kin_soup.find('div', class_="c-heading__content").text.replace("\n", "").replace("\t", "")
            except:
                kin_body="질문없음"

            # 답변 내용
            ans = ""
            try:
                for kin_ans in kin_soup.find_all('div', class_="se-component se-text se-l-default"):
                    txt=re.sub("[^가-힣 ]","",kin_ans.text)
                    ans = ans + txt +"\n"
            except:
                ans="답변없음"

            body_lst.append({"질문":kin_body, "답변":ans})
            
    df_body=pd.DataFrame(body_lst)
    #display(df_body.head(3))
    return df_body


In [5]:
encText = input("검색어 입력:")

df_navkin=pd.DataFrame()

for i in range(1, 101,100):

    df=get_Nav_client(encText, i)
    df_navkin=pd.concat([df_navkin, df])

df_navkin

검색어 입력:우울증


,title,link,description,질문,답변
0,<b>우울증</b> 질문,https://kin.naver.com/qna/detail.naver?d1id=7&...,제가 <b>우울증</b>이 있는 것 같아서 정신과에 다녀오고 약을... 이건 <b>...,제가 우울증이 있는 것 같아서 정신과에 다녀오고 약을 받았는데 친구랑 있으면 즐겁고...,안녕하세요 닥톡네이버 지식 상담한의사 맹아름입니다우울증은 단순히 우울한 감정...
1,<b>우울증</b> 환자 조루증 질문입니다.,https://kin.naver.com/qna/detail.naver?d1id=7&...,... <b>우울증</b>이 생긴 후 부터 심한 조루증이 생겼습니다. 아직... <...,심한 조루증 환자입니다. 우울증이 생긴 후 부터 심한 조루증이 생겼습니다. 아직 ...,안녕하세요 대한의사협회네이버 지식 상담의사 심영진 입니다우울증 등 정신 질환...
2,정신병원(<b>우울증</b>),https://kin.naver.com/qna/detail.naver?d1id=7&...,... 제가 인터넷에서 <b>우울증</b> 자가진단을 하였는데 30점이... 제대로...,전 중3입니다.제가 인터넷에서 우울증 자가진단을 하였는데 30점이 나왔어요.그래서 ...,안녕하세요 청소년사이버상담센터입니다 우울증 자가진단을 했는데 점이 나왔고부...
3,청소년 <b>우울증</b> 해결법,https://kin.naver.com/qna/detail.naver?d1id=6&...,제가 왜 <b>우울증</b>인지도 이젠 잘 모르겠어요 엄마아빠한텐 제가 죽을때까지 ...,제가 왜 우울증인지도 이젠 잘 모르겠어요엄마아빠한텐 제가 죽을때까지 절대 말 안할거...,안녕하세요저는 우울증 년간 겪고 정신과 치료 약물 치료 심리 치료 이외에 다...
4,비전형적 <b>우울증</b> 체중감소,https://kin.naver.com/qna/detail.naver?d1id=7&...,<b>우울증</b> 약을 이제 막 받아보기 시작한 20대 초반 여성인데요... 여러...,우울증 약을 이제 막 받아보기 시작한 20대 초반 여성인데요병원에 병명을 물어보아도...,안녕하세요 하이닥네이버 지식 상담의 한경호 입니다질문내용 잘 보았습니다답변드...
...,...,...,...,...,...
95,<b>우울증</b> 약 관련 질문드립니다,https://kin.naver.com/qna/detail.naver?d1id=7&...,<b>우울증</b> 약 관련 질문드립니다.. 지금 제몸 상태가 하루소주2병 마시고....,질문없음,약들은 우울증불면증에 먹는 약 맞네요저도 우울증 불안증 공황장애로 원인모르고...
96,<b>우울증</b>인가요?,https://kin.naver.com/qna/detail.naver?d1id=7&...,... 제가 느끼는 증상들이 <b>우울증</b>이 맞는지 알고 싶어서 질문합니다. ...,현재 대학에 재학중인 여자입니다. 제가 느끼는 증상들이 우울증이 맞는지 알고 싶어서...,무기력증이 심하신 것 같은데 자칫 잘못하면우울증 증상이 심해질 수 있으니 병...
97,<b>우울증</b>때문에 너무 힘들어요,https://kin.naver.com/qna/detail.naver?d1id=7&...,<b>우울증</b>때문에 주변사람들한테 너무 피해를 끼치고 하루하루가 살고싶지 않을...,질문없음,
98,이것도 <b>우울증</b>이에요???,https://kin.naver.com/qna/detail.naver?d1id=7&...,... 인터넷에 있는 <b>우울증</b> 테스트 해봤는데 <b>우울증</b>은 전혀...,낮에는 되게 재밌고 살고싶고 좋은데 침대에 자려고 누우면 계속 다음날에 제가 죽어있...,우울증 보다는 조울증 증상같아 보입니다정확한 진단을 위해 정신건강의학과를 내...


In [6]:
df_navkin.columns = ['제목', '링크', '요약', '질문', '답변']
df_navkin = df_navkin.reset_index(drop = True)
df_navkin.to_csv('./' + encText + "_지식인.csv", index = False)